# Binary Classification evaluation score re-assessment

Costa Rica

Belinda Brown, belindabrownr04@gmail.com

[![GitHub](https://badgen.net/badge/icon/github?icon=github&label)](https://github.com) [brown9804](https://github.com/brown9804)


Jan, 2021

## Import packages 

In [ ]:
!pip install prettytable

In [2]:
import os
import glob
import logging
import sys
import pandas as pd
import numpy as np
import openpyxl
from tqdm import tqdm
from datetime import date
import matplotlib.pyplot as plt  
import sklearn
from sklearn import preprocessing
from sklearn import metrics
from azureml.core import Workspace, Dataset
import time
import prettytable

## Functions Definitions

In [3]:
def xcolumnlookup(lookup_value, lookup_array, return_array, if_not_found:str = ''):
    match_value = return_array.loc[lookup_array == lookup_value]
    if match_value.empty:
        return "Not Found" if if_not_found == '' else if_not_found

    else:
        return match_value.tolist()[0]

In [4]:
def checkType(a_list):
    # timeout variable can be omitted, if you use specific value in the while condition
    timeout = 0.0000000056  # [seconds]
    timeout_start = time.time()
    while time.time() < timeout_start + timeout:
        for element in a_list:
            if isinstance(element, int):
                print("It's an Integer")
            if isinstance(element, str):
                print("It's an string")
            if isinstance(element, float):
                print("It's an floating number")

In [5]:
def analysis_percentil(dataframe, column_name):
    output_df = pd.DataFrame()   
    output_df['25%']  = [x if x <= 0.25 else '-' for x in dataframe[column_name]]
    output_df['50%']  = [x if x > 0.25 and x <= 0.50 else '-' for x in dataframe[column_name]]
    output_df['75%']  = [x if x > 0.50 and x <= 0.75 else '-' for x in dataframe[column_name]]
    output_df['100%']  = [x if x > 0.75 and x <= 100 else '-' for x in dataframe[column_name]]
    
    items25percentil = 0
    items50percentil = 0
    items75percentil = 0
    items100percentil = 0

    for i in output_df['25%']:
        if i != '-':
            items25percentil= items25percentil+ 1
    for i in output_df['50%']:
        if i != '-':
            items50percentil= items50percentil+ 1    
    for i in output_df['75%']:
        if i != '-':
            items75percentil= items75percentil+ 1        
    for i in output_df['100%']:
        if i != '-':
            items100percentil= items100percentil+ 1   

    ip0 = str(items25percentil)
    ip1 = str(items50percentil)
    ip2 = str(items75percentil)
    ip3 = str(items100percentil)
    p1 = str(items25percentil/output_df.shape[0]*100)
    p2 = str(items50percentil/output_df.shape[0]*100)
    p3 = str(items75percentil/output_df.shape[0]*100)
    p4 = str(items100percentil/output_df.shape[0]*100)
    return output_df, ip0, ip1, ip2, ip3, p1, p2, p3, p4

In [6]:
def filter_unique(dataframe, column, newcolumn):
    count_unique = 0
    output_df = pd.DataFrame()
    dataframe = dataframe[column]
    print("Total of items: ", len(dataframe))
    # using set()
    # to remove duplicated 
    # from list 
    output_df = list(set(dataframe))
    count_unique = len(output_df)
    print("Unique items: ", count_unique)
    unique_column = pd.Series(output_df,name=newcolumn)
    return output_df, count_unique, unique_column

In [7]:
def mapping_data(dataframe):
    head = pd.DataFrame(dataframe.iloc[:3])
    display(head)
    
    output = {}
    output['Column Names'] = dataframe.columns.values.tolist()
    pd.set_option('display.max_colwidth', -1)
    column_names = pd.DataFrame(data=output)
    display(column_names)
    
    output = {}
    output['Data Types'] = dataframe.dtypes
    pd.set_option('display.max_colwidth', -1) 
    column_dtypes = pd.DataFrame(data=output)
    display(column_dtypes)
    
    output = {}
    output['Null Values'] = dataframe.isna().sum()  # #  Is null # this is to validate the amount of rows within a columns that carry information
    pd.set_option('display.max_colwidth', -1) 
    null_values = pd.DataFrame(data=output)
    display(null_values)

In [8]:
def reading_xlsx_data(filename):
    # Define variable to load the wookbook
    wookbook = openpyxl.load_workbook(filename)
    # Define variable to read the active sheet:
    ws = wookbook.active
    data = ws.values
    # Get the first line in file as a header line
    columns = next(data)[0:]
    # Create a DataFrame based on the second and subsequent lines of data
    source_df = pd.DataFrame(data, columns=columns)
    return source_df

In [9]:
def snapshot_by_equal_Column_value(dataframe, column_name, value):
    output_df = pd.DataFrame()
    output_df = dataframe.loc[dataframe[column_name] == value]
    display(output_df.head(3))
    return output_df

In [10]:
def snapshot_by_less_Column_value(dataframe, column_name, value):
    output_df = pd.DataFrame()
    output_df = dataframe.loc[dataframe[column_name] < value]
    display(output_df.head(3))
    return output_df

In [11]:
def snapshot_by_greater_equal_Column_value(dataframe, column_name, value):
    output_df = pd.DataFrame()
    output_df = dataframe.loc[dataframe[column_name] >= value]
    display(output_df.head(3))
    return output_df

## Reading source data 

In [12]:
source_df = reading_xlsx_data("materials_data_evaluation.xlsx")

## Mapping 

In [13]:
mapping_data(source_df)

,Code,Item,Binary classification,Matching result,Item Classification,Evaluation score,Next Stage,ID,ID Description
0,309.0,8.0,INVALID,True,Solid,0.5244,Discovery,8.235768e+10,Solid309-8
1,317.0,16.0,INVALID,True,Solid,0.983,Discovery,8.235768e+10,Solid317-16
2,325.0,24.0,INVALID,True,Solid,0.5642,Discovery,8.235768e+10,Solid325-24


,Column Names
0,Code
1,Item
2,Binary classification
3,Matching result
4,Item Classification
5,Evaluation score
6,Next Stage
7,ID
8,ID Description


,Data Types
Code,float64
Item,float64
Binary classification,object
Matching result,object
Item Classification,object
Evaluation score,object
Next Stage,object
ID,float64
ID Description,object


,Null Values
Code,1035188
Item,1035188
Binary classification,1035188
Matching result,1035188
Item Classification,0
Evaluation score,0
Next Stage,1035188
ID,1035188
ID Description,1035188


In [14]:
source_df = source_df.astype({'Evaluation score':'float64'})

In [15]:
output = {}
output['Source Data'] = source_df.dtypes
pd.set_option('display.max_colwidth', -1) 
pd.DataFrame(data=output)

,Source Data
Code,float64
Item,float64
Binary classification,object
Matching result,object
Item Classification,object
Evaluation score,float64
Next Stage,object
ID,float64
ID Description,object


In [16]:
source_df.shape

(1048575, 9)

### **Cleaning up**

In [17]:
source_df = source_df.drop_duplicates(subset="ID")

In [18]:
source_df.shape

(13387, 9)

## Snapshot for the analysis

### **Matching Results**

In [19]:
matching_results = snapshot_by_equal_Column_value(source_df,'Matching result', True)

,Code,Item,Binary classification,Matching result,Item Classification,Evaluation score,Next Stage,ID,ID Description
0,309.0,8.0,INVALID,True,Solid,0.5244,Discovery,8.235768e+10,Solid309-8
1,317.0,16.0,INVALID,True,Solid,0.9830,Discovery,8.235768e+10,Solid317-16
2,325.0,24.0,INVALID,True,Solid,0.5642,Discovery,8.235768e+10,Solid325-24


### **No Matching Results**

In [20]:
no_matching_results = snapshot_by_equal_Column_value(source_df,'Matching result', False)

,Code,Item,Binary classification,Matching result,Item Classification,Evaluation score,Next Stage,ID,ID Description
6,357.0,56.0,VALID,False,Solid,0.668024,Discovery,8.235768e+10,Solid357-56
7,371.0,64.0,INVALID,False,Solid,0.577878,Discovery,8.235768e+10,Solid371-64
8,379.0,72.0,VALID,False,Solid,0.734285,Discovery,8.235768e+10,Solid379-72


### **Items < Threshold**

In [21]:
less_thr = snapshot_by_less_Column_value(source_df,'Evaluation score', 0.80)

,Code,Item,Binary classification,Matching result,Item Classification,Evaluation score,Next Stage,ID,ID Description
0,309.0,8.0,INVALID,True,Solid,0.5244,Discovery,8.235768e+10,Solid309-8
2,325.0,24.0,INVALID,True,Solid,0.5642,Discovery,8.235768e+10,Solid325-24
4,341.0,40.0,INVALID,True,Solid,0.7733,Discovery,8.235768e+10,Solid341-40


### **Items >= Threshold**

In [22]:
greater_thr = snapshot_by_greater_equal_Column_value(source_df,'Evaluation score', 0.80)

,Code,Item,Binary classification,Matching result,Item Classification,Evaluation score,Next Stage,ID,ID Description
1,317.0,16.0,INVALID,True,Solid,0.983000,Discovery,8.235768e+10,Solid317-16
3,333.0,32.0,INVALID,True,Solid,0.980300,Discovery,8.235768e+10,Solid333-32
5,349.0,48.0,INVALID,True,Solid,0.980366,Discovery,8.235768e+10,Solid349-48


### **Matching + Items >= Threshold**

In [23]:
mathing_greater_thr = snapshot_by_greater_equal_Column_value(matching_results,'Evaluation score', 0.80 )

,Code,Item,Binary classification,Matching result,Item Classification,Evaluation score,Next Stage,ID,ID Description
1,317.0,16.0,INVALID,True,Solid,0.983000,Discovery,8.235768e+10,Solid317-16
3,333.0,32.0,INVALID,True,Solid,0.980300,Discovery,8.235768e+10,Solid333-32
5,349.0,48.0,INVALID,True,Solid,0.980366,Discovery,8.235768e+10,Solid349-48


### **Matching + Items < Threshold**

In [24]:
mathing_less_thr = snapshot_by_less_Column_value(matching_results,'Evaluation score', 0.80 )

,Code,Item,Binary classification,Matching result,Item Classification,Evaluation score,Next Stage,ID,ID Description
0,309.0,8.0,INVALID,True,Solid,0.5244,Discovery,8.235768e+10,Solid309-8
2,325.0,24.0,INVALID,True,Solid,0.5642,Discovery,8.235768e+10,Solid325-24
4,341.0,40.0,INVALID,True,Solid,0.7733,Discovery,8.235768e+10,Solid341-40


## Analysis

In [25]:
# Shapes
output = {}
output['Description'] = 'Shapes'
output['Source Data'] = str(source_df.shape)
output['Matching Results'] = str(matching_results.shape)
output['No Matching Results'] = str(no_matching_results.shape)
output['Less Threshold'] = str(less_thr.shape)
output['Greater Threshold'] = str(greater_thr.shape)
output['Matching + Greater Thr Results'] = str(mathing_greater_thr.shape)
output['Matching + Less Thr Results'] = str(mathing_less_thr.shape)
pd.set_option('display.max_colwidth', -1) 
pd.DataFrame(data=output, index=['']).T

,
Description,Shapes
Source Data,"(13387, 9)"
Matching Results,"(10924, 9)"
No Matching Results,"(2462, 9)"
Less Threshold,"(6060, 9)"
Greater Threshold,"(7327, 9)"
Matching + Greater Thr Results,"(6896, 9)"
Matching + Less Thr Results,"(4028, 9)"


In [26]:
# Percentage 
output = {}
output['Description'] = 'Percentage'
output['Source Data'] = str(source_df.shape[0]/source_df.shape[0]*100)
output['Matching Results'] = str(matching_results.shape[0]/source_df.shape[0]*100)
output['No Matching Results'] = str(no_matching_results.shape[0]/source_df.shape[0]*100)
output['Less Threshold'] = str(less_thr.shape[0]/source_df.shape[0]*100)
output['Greater Threshold'] = str(greater_thr.shape[0]/source_df.shape[0]*100)
output['Matching + Greater Thr Results'] = str(mathing_greater_thr.shape[0]/source_df.shape[0]*100)
output['Matching + Less Thr Results'] = str(mathing_less_thr.shape[0]/source_df.shape[0]*100)
pd.set_option('display.max_colwidth', -1) 
pd.DataFrame(data=output, index=['']).T

,
Description,Percentage
Source Data,100.0
Matching Results,81.60155374617166
No Matching Results,18.39097632031075
Less Threshold,45.267797116605664
Greater Threshold,54.73220288339434
Matching + Greater Thr Results,51.51266153731232
Matching + Less Thr Results,30.088892208859342


### **Unique Items**

In [27]:
UniqueItems, len_UniqueItems, x0 = filter_unique(source_df, 'ID', 'Unique Items')

Total of items:  13387
Unique items:  13387


### **Evaluation score**

In [28]:
UniqueScore, len_UniqueScore, x1 = filter_unique(source_df, 'Evaluation score', 'Unique Score')

Total of items:  13387
Unique items:  13360


In [29]:
checkType(UniqueScore)

In [30]:
analyzed_Data = pd.concat([x0,x1], axis=1)

In [31]:
analyzed_Data.head(2)

,Unique Items,Unique Score
0,NaN,0.524400
1,8.235771e+10,0.908806


#### Ranges

In [32]:
percentages, ip0, ip1, ip2, ip3, p1, p2, p3, p4 = analysis_percentil(analyzed_Data,'Unique Score')

In [33]:
percentages.head(10)

,25%,50%,75%,100%
0,-,-,0.5244,-
1,-,-,-,0.908806
2,-,-,-,0.884479
3,-,-,0.552988,-
4,-,-,0.536063,-
5,-,-,-,0.989085
6,-,-,-,0.876486
7,-,-,-,0.933805
8,-,-,0.568884,-
9,-,-,-,0.928909


In [34]:
percentages.shape

(13387, 4)

In [35]:
output = {}
output['Description'] = ['Items in 25% range', 'Items in 50% range', 'Items in 75% range', 'Items in 100% range', '0%-25% percentil represents', '25%-50% percentil represents', '50%-75% percentil represents', '75%-100% percentil represents']
output['Represents'] = [ip0, ip1, ip2, ip3, p1, p2, p3, p4]
pd.set_option('display.max_colwidth', -1) 
pd.DataFrame(data=output)

,Description,Represents
0,Items in 25% range,1
1,Items in 50% range,402
2,Items in 75% range,4780
3,Items in 100% range,8177
4,0%-25% percentil represents,0.007469933517591694
5,25%-50% percentil represents,3.0029132740718607
6,50%-75% percentil represents,35.7062822140883
7,75%-100% percentil represents,61.081646373347276


In [ ]:
items_result_analysis_0 = [ip0, ip1, ip2, ip3]
percentil_result_analysis_0 = [ p1, p2, p3, p4]

In [36]:
percentages2, ip00, ip11, ip22, ip33, p11, p22, p33, p44 = analysis_percentil(mathing_less_thr,'Evaluation score')

In [37]:
output = {}
output['Description'] = ['Items in 25% range', 'Items in 50% range', 'Items in 75% range', 'Items in 100% range', '0%-25% percentil represents', '25%-50% percentil represents', '50%-75% percentil represents', '75%-100% percentil represents']
output['Represents'] = [ip00, ip11, ip22, ip33, p11, p22, p33, p44]
pd.set_option('display.max_colwidth', -1) 
pd.DataFrame(data=output)

,Description,Represents
0,Items in 25% range,0
1,Items in 50% range,199
2,Items in 75% range,3146
3,Items in 100% range,683
4,0%-25% percentil represents,0.0
5,25%-50% percentil represents,4.940417080436942
6,50%-75% percentil represents,78.10327706057598
7,75%-100% percentil represents,16.95630585898709


In [ ]:
items_result_analysis_1 = [ip00, ip11, ip22, ip33]
percentil_result_analysis_1 = [ p11, p22, p33, p44]

In [38]:
source_df.groupby(["Item Classification"])["Binary classification"].value_counts(normalize=True).unstack(fill_value=0) 

Binary classification,INVALID,VALID
Item Classification,,
Colloidal,1.000000,0.000000
Gas,0.554726,0.445274
Liquid,0.802623,0.197377
Solid,0.163049,0.836951


In [39]:
source_df.groupby(["Item Classification"])["Matching result"].value_counts(normalize=True).unstack(fill_value=0) 

Matching result,False,True
Item Classification,,
Colloidal,0.000000,1.000000
Gas,0.446517,0.553483
Liquid,0.192380,0.807620
Solid,0.163688,0.836312


In [40]:
source_df.groupby(["Next Stage"])["Binary classification"].value_counts(normalize=True).unstack(fill_value=0)

Binary classification,INVALID,VALID
Next Stage,,
Discovery,0.264303,0.735697
Plan,0.000000,1.000000


In [41]:
source_df.groupby(["Next Stage"])["Matching result"].value_counts(normalize=True).unstack(fill_value=0)

Matching result,False,True
Next Stage,,
Discovery,0.18413,0.81587
Plan,0.00000,1.00000


In [42]:
source_df.groupby(["Matching result"])["Code"].value_counts(normalize=True).unstack(fill_value=0) 

Code,285.0,309.0,317.0,325.0,333.0,341.0,349.0,357.0,371.0,379.0,...,116225.0,116233.0,116241.0,116249.0,116257.0,116265.0,116279.0,116287.0,116295.0,116303.0
Matching result,,,,,,,,,,,,,,,,,,,,,
False,0.000406,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000406,0.000406,0.000406,...,0.000000,0.000406,0.000406,0.000000,0.000406,0.000000,0.000406,0.000406,0.000000,0.000000
True,0.000000,0.000092,0.000092,0.000092,0.000092,0.000092,0.000092,0.000000,0.000000,0.000000,...,0.000092,0.000000,0.000000,0.000092,0.000000,0.000092,0.000000,0.000000,0.000092,0.000092


In [48]:
source_df.groupby(["Matching result"])["Code"].value_counts(normalize=True).unstack(fill_value=0) 

Code,285.0,309.0,317.0,325.0,333.0,341.0,349.0,357.0,371.0,379.0,...,116225.0,116233.0,116241.0,116249.0,116257.0,116265.0,116279.0,116287.0,116295.0,116303.0
Matching result,,,,,,,,,,,,,,,,,,,,,
False,0.000406,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000406,0.000406,0.000406,...,0.000000,0.000406,0.000406,0.000000,0.000406,0.000000,0.000406,0.000406,0.000000,0.000000
True,0.000000,0.000092,0.000092,0.000092,0.000092,0.000092,0.000092,0.000000,0.000000,0.000000,...,0.000092,0.000000,0.000000,0.000092,0.000000,0.000092,0.000000,0.000000,0.000092,0.000092


Can be used with group by:

In [ ]:
gb.agg        gb.boxplot    gb.cummin     gb.describe   gb.filter     gb.get_group  gb.height     gb.last       gb.median     gb.ngroups    gb.plot       gb.rank       gb.std        gb.transform
gb.aggregate  gb.count      gb.cumprod    gb.dtype      gb.first      gb.groups     gb.hist       gb.max        gb.min        gb.nth        gb.prod       gb.resample   gb.sum        gb.var
gb.apply      gb.cummax     gb.cumsum     gb.fillna     gb.gender     gb.head       gb.indices    gb.mean       gb.name       gb.ohlc       gb.quantile   gb.size       gb.tail       gb.weight

In [53]:
source_df_with_median = source_df.groupby(['Item','ID','Binary classification']).agg(arg={'Binary classification':[('_count','count')],'Evaluation score':[('','median')]}).reset_index()

In [54]:
source_df_with_max = source_df.groupby(['Item','ID','Binary classification']).agg(arg={'Binary classification':[('_count','count')],'Evaluation score':[('','max')]}).reset_index()

In [55]:
source_df_with_median.head(3)

Item            ID Binary classification        Evaluation score
                                            _count                 
0  8.0   8.235768e+10  INVALID               1      0.5244         
1  16.0  8.235768e+10  INVALID               1      0.9830         
2  24.0  8.235768e+10  INVALID               1      0.5642

In [56]:
source_df_with_max.head(3)

Item            ID Binary classification        Evaluation score
                                            _count                 
0  8.0   8.235768e+10  INVALID               1      0.5244         
1  16.0  8.235768e+10  INVALID               1      0.9830         
2  24.0  8.235768e+10  INVALID               1      0.5642

In [ ]:
# Export to CSV
source_df.to_csv('./Exporting_DataFrame.csv', index=False)    

In [ ]:
source_df['Solid'] = np.where(source_df.astype(str).apply(lambda x: x.str.contains('Solid').any(), axis=1), 'no', 'yes')

In [ ]:
plt.figure(figsize=(15,8))
plt.title('Correlation Analysis') # distance 
plt.xlabel('Percentil')
ax1 = percentil_result_analysis_0.inner_open_corr.plot(color='blue', grid=True, label='0 - analysis') # dataframe if not wk
ax2 = percentil_result_analysis_1.inner_closed_corr.plot(color='red', grid=True, secondary_y=True, label='1 - analysis')# dataframe if not wk
h1, l1 = ax1.get_legend_handles_labels()
h2, l2 = ax2.get_legend_handles_labels()
plt.legend(h1+h2, l1+l2, loc=2)
plt.show()

In [ ]:
plt.scatter(items_result_analysis_0,items_result_analysis_1, label='items comparison',color='g')
plt.title('Items - Correlation Analysis') # distance 
plt.xlabel('Analysis results')
plt.ylabel('Nunber of items')
plt.legend()
plt.show()

# END